In [1]:
import pandas as pd
import yfinance as yf
import talib 

PATH = "/Users/benpfeffer/trading-icona-capital/trading-icona-capital/src/"

In [2]:
tickers = [
    "MC.PA", "OR.PA", "TTE.PA", "SAN.PA", "AIR.PA", "SU.PA", "AI.PA", "RMS.PA", "SAF.PA", "EL.PA",
    "BNP.PA", "CS.PA", "DG.PA", "SGO.PA", "BN.PA", "RI.PA", "KER.PA",
    "SAP.DE", "SIE.DE", "DTE.DE", "ALV.DE", "MUV2.DE", "DB1.DE", "IFX.DE", "ADS.DE", "BAS.DE",
    "MBG.DE", "DHL.DE", "BMW.DE", "BAYN.DE", "VOW3.DE",
    "ASML.AS", "PRX.AS", "INGA.AS", "ADYEN.AS", "WKL.AS", "AD.AS",
    "IBE.MC", "SAN.MC", "ITX.MC", "BBVA.MC",
    "ISP.MI", "UCG.MI", "ENEL.MI", "RACE.MI", "ENI.MI", "STLAM.MI",
    "ABI.BR",
    "NDA-FI.HE", "NOKIA.HE"
]
# cross check we have 50 tickers [EUROSTOXX 50]
assert(len(tickers) == 50)

In [3]:
# Download historical data (last 3 years for example)
data = {}
for ticker in tickers:
    df = yf.download(ticker, period="3y", interval="1d")
    df.dropna(inplace=True)
    data[ticker] = df

# Example: view data for one ticker
print(data['ADS.DE'].head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Price            Close        High         Low        Open  Volume
Ticker          ADS.DE      ADS.DE      ADS.DE      ADS.DE  ADS.DE
Date                                                              
2022-02-04  234.009262  237.615399  233.180819  236.640767  702827
2022-02-07  234.350357  236.202168  233.326990  236.055964  513006
2022-02-08  233.034622  234.935152  229.818332  234.350367  763011
2022-02-09  238.736221  239.808323  234.983895  235.130084  496452
2022-02-10  236.738220  241.806321  234.691501  239.710852  404418


In [4]:
data

{'MC.PA': Price            Close        High         Low        Open  Volume
 Ticker           MC.PA       MC.PA       MC.PA       MC.PA   MC.PA
 Date                                                              
 2022-02-04  669.843872  676.674185  663.393043  672.974473  406201
 2022-02-07  679.330444  682.176404  667.092854  673.543684  353733
 2022-02-08  669.843872  686.350458  667.187655  679.330402  421466
 2022-02-09  679.804749  683.694261  674.966641  676.389621  460237
 2022-02-10  669.938843  680.848354  661.685549  679.615116  380675
 ...                ...         ...         ...         ...     ...
 2025-01-29  713.200012  722.200012  700.000000  720.500000  984398
 2025-01-30  715.799988  718.900024  710.799988  718.400024  420057
 2025-01-31  703.500000  718.000000  698.799988  716.900024  578611
 2025-02-03  690.099976  693.799988  683.299988  685.099976  695311
 2025-02-04  692.700012  695.299988  682.900024  692.299988  370853
 
 [768 rows x 5 columns],
 'OR.PA': Pr

In [5]:
def compute_moving_avg(df:pd.DataFrame, window_a:int=50, window_b:int=200) -> pd.Series:
    """
    Description:
    Compute two moving averages:
    - window_a: 50-day moving average
    - window_b: 200-day moving average
    - add them to the dataframe

    Input:
    - pandas DataFrame
    - window_a: int
    - window_b: int

    Output:
    - pandas DataFrame with the moving averages added

    Context:
    There are two main types: Simple Moving Average (SMA) and Exponential Moving Average (EMA). 
    SMA is straightforward—it's just the average of closing prices over a certain period. 
    For   example, a 20-day SMA would add up the last 20 closing prices and divide by 20. EMA is a bit more complex because it gives more weight to recent prices. 
    The formula for EMA involves a smoothing factor, usually calculated as 2/(n+1) where n is the period. So, each day's EMA is previous EMA plus smoothing factor times (current price minus previous EMA). Wait, I need to make sure I get the EMA formula right. Maybe it's better to write it step by step.
    """

    # Simple Moving Average

    # using the formula SMA = Sum(P_i) / n
    # where P_i is the closing price of the i-th day and n is the number of days
    # we can use the pandas rolling method to compute the SMA
    df['SMA_50'] = df['Close'].rolling(window=window_a).mean()
    df['SMA_200'] = df['Close'].rolling(window=window_b).mean()

    # Exponential Moving Average
    
    # using the formula EMA = (P_t - EMA_{t-1}) * (2 / (n + 1)) + EMA_{t-1}
    # where P_t is the closing price of the t-th day and n is the number of days
    # we can use the pandas ewm method to compute the EMA
    df['EMA_50'] = df['Close'].ewm(span=window_a, adjust=False).mean()
    df['EMA_200'] = df['Close'].ewm(span=window_b, adjust=False).mean()

    # Now let's add another column that will be used to detect where the 200-day EMA is above the 50-day EMA
    # This will be used to identify the trending regime
    df['trending'] = df['EMA_200'] > df['EMA_50']

    # Now let's add another column that will be used to detect where the 200-day EMA is below the 50-day EMA
    # This will be used to identify the range-bound regime
    df['range_bound'] = df['EMA_200'] < df['EMA_50']
    
    return df





In [6]:
def compute_macd(df:pd.DataFrame, window_fast:int=12, window_slow:int=26, window_signal:int=9) -> pd.Series:
    """
    """
    """
    Compute the Moving Average Convergence Divergence (MACD) indicator.
    
    Parameters:
    - df: pandas DataFrame containing price data
    - window_fast: int, period for fast EMA (default 12)
    - window_slow: int, period for slow EMA (default 26) 
    - window_signal: int, period for signal line EMA (default 9)
    
    Returns:
    - pandas DataFrame with MACD indicators added
    """
    
    # Calculate fast and slow EMAs
    ema_fast = df['Close'].ewm(span=window_fast, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=window_slow, adjust=False).mean()
    
    # Calculate MACD line
    df['MACD_line'] = ema_fast - ema_slow
    
    # Calculate Signal line (9-day EMA of MACD line)
    df['MACD_signal'] = df['MACD_line'].ewm(span=window_signal, adjust=False).mean()
    
    # Calculate MACD histogram
    df['MACD_histogram'] = df['MACD_line'] - df['MACD_signal']
    
    # add the columns to the dataframe
    df['MACD_line'] = df['MACD_line']
    df['MACD_signal'] = df['MACD_signal']
    df['MACD_histogram'] = df['MACD_histogram']

    return df

In [7]:
def compute_rsi(df: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    """
    Compute the Relative Strength Index (RSI) indicator.
    
    Parameters:
    - df: pandas DataFrame containing price data
    - window: int, lookback period for RSI calculation (default 14)
    
    Returns:
    - pandas DataFrame with RSI indicator added
    """
    # Calculate price changes
    delta = df['Close'].diff()
    
    # Separate gains and losses
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses
    avg_gains = gains.rolling(window=window, min_periods=1).mean()
    avg_losses = losses.rolling(window=window, min_periods=1).mean()
    
    # Calculate RS and RSI
    rs = avg_gains / avg_losses
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Calculate True Range
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift())
    low_close = abs(df['Low'] - df['Close'].shift())
    
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(window=window).mean()
    
    # Calculate Variance
    df['Variance'] = df['Close'].rolling(window=window).var()


    return df

In [8]:
def compute_keltern_channel(df: pd.DataFrame, window: int = 20, multiplier: float = 2.0) -> pd.DataFrame:
    """
    Compute Keltner Channels with 3 layers.
    
    Parameters:
    - df: pandas DataFrame containing price data
    - window: int, lookback period for EMA calculation (default 20)
    - multiplier: float, multiplier for ATR bands (default 2.0)
    
    Returns:
    - pandas DataFrame with Keltner Channel indicators added
    """
    # Calculate EMA of typical price
    df['KC_middle'] = df['Close'].ewm(span=window, adjust=False).mean()
    
    # Calculate outer bands using ATR
    df['KC_upper3'] = df['KC_middle'] + (multiplier * 1.5 * df['ATR'])
    df['KC_upper2'] = df['KC_middle'] + (multiplier * 1.0 * df['ATR']) 
    df['KC_upper1'] = df['KC_middle'] + (multiplier * 0.5 * df['ATR'])
    
    df['KC_lower1'] = df['KC_middle'] - (multiplier * 0.5 * df['ATR'])
    df['KC_lower2'] = df['KC_middle'] - (multiplier * 1.0 * df['ATR'])
    df['KC_lower3'] = df['KC_middle'] - (multiplier * 1.5 * df['ATR'])
    
    return df

In [9]:
def compute_std_dev(df: pd.DataFrame, window: int = 20) -> pd.DataFrame:
    """
    Compute rolling standard deviation of closing prices.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing price data
    window : int, optional
        Rolling window size, by default 20
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with standard deviation indicator added
    """
    df['RSTD'] = df['Close'].rolling(window=window).std()
    return df


In [15]:
# 
df = compute_moving_avg(df)
df = compute_macd(df)
df = compute_rsi(df)
df = compute_std_dev(df)
df = compute_keltern_channel(df)

In [16]:
import os
import pandas as pd
from pathlib import Path

def compute_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute all technical indicators for a given DataFrame
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing price data
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with all indicators added
    """
    df = compute_moving_avg(df)
    df = compute_macd(df)
    df = compute_rsi(df)
    df = compute_keltern_channel(df)
    return df

def process_and_save_stocks(cache_dir: Path, output_dir: Path) -> None:
    """
    Process stock data with technical indicators and save results
    
    Parameters:
    -----------
    cache_dir : Path
        Directory containing cached stock data files
    output_dir : Path
        Directory to save processed data with indicators
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for file in cache_dir.glob('*.parquet'):
        try:
            df = pd.read_parquet(file)
            df = compute_indicators(df)
            
            output_file = output_dir / f"{file.stem}_indicators.parquet"
            df.to_parquet(output_file)
            print(f"Processed and saved {file.name}")
            
        except Exception as e:
            print(f"Error processing {file.name}: {str(e)}")
            
    print("Processing complete!")

# Set paths
CACHE_DIR = Path('/Users/benpfeffer/trading-icona-capital/trading-icona-capital/src/db/cache')
OUTPUT_DIR = CACHE_DIR.parent / 'with_indicators'

# Run processing
process_and_save_stocks(CACHE_DIR, OUTPUT_DIR)


Processed and saved AD_AS.parquet
Processed and saved STLAM_MI.parquet
Processed and saved ABI_BR.parquet
Processed and saved WKL_AS.parquet
Processed and saved AIR_PA.parquet
Processed and saved SIE_DE.parquet
Processed and saved DB1_DE.parquet
Processed and saved CS_PA.parquet
Processed and saved NDA-FI_HE.parquet
Processed and saved TTE_PA.parquet
Processed and saved PRX_AS.parquet
Processed and saved IBE_MC.parquet
Processed and saved MC_PA.parquet
Processed and saved BAS_DE.parquet
Processed and saved KER_PA.parquet
Processed and saved IFX_DE.parquet
Processed and saved ALV_DE.parquet
Processed and saved RI_PA.parquet
Processed and saved DTE_DE.parquet
Processed and saved ISP_MI.parquet
Processed and saved UCG_MI.parquet
Processed and saved ENEL_MI.parquet
Processed and saved BNP_PA.parquet
Processed and saved SAP_DE.parquet
Processed and saved MUV2_DE.parquet
Processed and saved SAN_MC.parquet
Processed and saved RMS_PA.parquet
Processed and saved SGO_PA.parquet
Processed and sav

In [17]:
df_ =pd.read_parquet('/Users/benpfeffer/trading-icona-capital/trading-icona-capital/src/db/with_indicators/ABI_BR_indicators.parquet')
df_

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_50,SMA_200,EMA_50,...,RSI,ATR,Variance,KC_middle,KC_upper3,KC_upper2,KC_upper1,KC_lower1,KC_lower2,KC_lower3
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-06 00:00:00+01:00,65.930155,66.677535,65.769321,66.119362,1454607,0.0,0.0,NaN,NaN,66.119362,...,NaN,NaN,NaN,66.119362,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-07 00:00:00+01:00,65.750401,65.977450,64.236724,64.804352,2098069,0.0,0.0,NaN,NaN,66.067793,...,0.000000,NaN,NaN,65.994123,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-10 00:00:00+01:00,64.047515,64.728672,63.829928,64.548927,1229569,0.0,0.0,NaN,NaN,66.008230,...,0.000000,NaN,NaN,65.856485,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-11 00:00:00+01:00,64.652990,64.700296,63.933998,64.066444,1775645,0.0,0.0,NaN,NaN,65.932081,...,0.000000,NaN,NaN,65.686005,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-12 00:00:00+01:00,64.293482,64.596217,64.075895,64.350243,1685676,0.0,0.0,NaN,NaN,65.870048,...,12.145175,NaN,NaN,65.558790,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-28 00:00:00+01:00,47.400002,48.160000,47.400002,47.439999,2456391,0.0,0.0,49.1114,54.903223,49.151744,...,50.273204,0.886428,0.779098,47.093094,49.752379,48.865950,47.979522,46.206666,45.320237,44.433809
2025-01-29 00:00:00+01:00,47.169998,47.580002,46.799999,47.189999,2268256,0.0,0.0,48.9916,54.872340,49.074813,...,47.074827,0.889286,0.706495,47.102323,49.770180,48.880894,47.991609,46.213037,45.323751,44.434466
2025-01-30 00:00:00+01:00,47.270000,47.990002,47.090000,47.840000,1641181,0.0,0.0,48.8808,54.840765,49.026389,...,68.159185,0.800714,0.832721,47.172578,49.574721,48.774007,47.973292,46.371863,45.571149,44.770434


In [21]:
import matplotlib.pyplot as plt
import seaborn as sns

def create_keltner_channel_plot(ax, df, color, level, alpha=0.5, fill_alpha=0.1):
    """Create a Keltner Channel plot with upper and lower bands"""
    ax.plot(df.index, df[f'KC_upper{level}'], f'{color}--', 
            label=f'KC Upper {level}', alpha=alpha)
    ax.plot(df.index, df[f'KC_lower{level}'], f'{color}--', 
            label=f'KC Lower {level}', alpha=alpha)
    ax.fill_between(df.index, df[f'KC_upper{level}'], df[f'KC_lower{level}'], 
                    alpha=fill_alpha, color=color)

def plot_price_and_keltner(ax, df):
    """Plot price data with Keltner Channels"""
    ax.plot(df.index, df['Close'], label='Close Price', alpha=0.7)
    
    # Plot Keltner Channels for 3 levels
    colors = ['r', 'b', 'g']
    for level, color in enumerate(colors, 1):
        create_keltner_channel_plot(ax, df, color, level)
    
    ax.set_title('Price & Keltner Channels')
    ax.legend(loc='upper left')
    ax.grid(True)

def plot_macd(ax, df):
    """Plot MACD indicator"""
    ax.plot(df.index, df['MACD_line'], label='MACD')
    ax.plot(df.index, df['MACD_signal'], label='Signal')
    ax.bar(df.index, df['MACD_histogram'], label='Histogram', alpha=0.3)
    ax.set_title('MACD')
    ax.legend(loc='upper left')
    ax.grid(True)

def plot_rsi(ax, df):
    """Plot RSI indicator with overbought/oversold levels"""
    ax.plot(df.index, df['RSI'], label='RSI')
    ax.axhline(y=70, color='r', linestyle='--', alpha=0.5)
    ax.axhline(y=30, color='g', linestyle='--', alpha=0.5)
    ax.set_title('RSI')
    ax.legend(loc='upper left')
    ax.grid(True)

def plot_moving_averages(ax, df):
    """Plot moving averages with market condition highlighting"""
    # Plot moving averages
    for period in [50, 200]:
        ax.plot(df.index, df[f'SMA_{period}'], label=f'{period}-day MA', alpha=0.7)
        ax.plot(df.index, df[f'EMA_{period}'], label=f'{period}-day EMA', alpha=0.7)
    
    # Market condition highlighting
    conditions = {
        'trending': ('limegreen', 'Trending'),
        'range_bound': ('orange', 'Range Bound'),
        'neither': ('red', 'Neither')
    }
    
    for idx in df.index:
        if df.at[idx, 'trending']:
            condition = 'trending'
        elif df.at[idx, 'range_bound']:
            condition = 'range_bound'
        else:
            condition = 'neither'
            
        color, label = conditions[condition]
        ax.axvspan(idx, idx + pd.Timedelta(days=1), 
                  color=color, alpha=0.2,
                  label=label if idx == df.index[0] else "")
    
    ax.set_title('Moving Averages')
    ax.legend(loc='upper left')
    ax.grid(True)

def plot_atr_variance(ax, df):
    """Plot ATR and Variance"""
    ax1 = ax
    ax2 = ax1.twinx()
    
    # Plot ATR
    ax1.plot(df.index, df['ATR'], 'b-', label='ATR')
    ax1.set_ylabel('ATR', color='b')
    
    # Calculate and plot rolling variance of close prices
    variance = df['Close'].rolling(window=20).var()
    ax2.plot(df.index, variance, 'r-', label='Variance (20-day)')
    ax2.set_ylabel('Variance', color='r')
    
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    ax1.grid(True)
    ax.set_title('ATR & Variance')

def plot_correlation_matrix(fig, df):
    """Create correlation matrix of indicators"""
    # Select relevant columns for correlation
    cols = ['Close', 'RSI', 'MACD_line', 'ATR', 'SMA_50', 'EMA_50', 
            'KC_upper1', 'KC_lower1', 'KC_upper2', 'KC_lower2', 
            'KC_upper3', 'KC_lower3', 'Variance', "ATR", 
            "MACD_histogram", "MACD_signal", "MACD_line"]
    corr = df[cols].corr()
    
    # Create correlation heatmap
    ax = plt.gca()
    sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax, fmt='.2f')
    ax.set_title('Correlation Matrix')

def plot_rstd(ax, df):
    """Plot Rolling Standard Deviation (RSTD)"""

    # Plot RSTD
    ax.plot(df.index, df['RSTD'], 'g-', label='Rolling Std Dev (20-day)')
    
    # Add horizontal lines at +/- 2 standard deviations from mean
    mean_rstd = df['RSTD'].mean()
    std_rstd = df['RSTD'].std()
    ax.axhline(y=mean_rstd + 2*std_rstd, color='r', linestyle='--', 
               alpha=0.5, label='+2σ')
    ax.axhline(y=mean_rstd - 2*std_rstd, color='r', linestyle='--', 
               alpha=0.5, label='-2σ')
    
    ax.set_ylabel('Standard Deviation')
    ax.set_title('Rolling Standard Deviation (20-day)')
    ax.legend(loc='upper left')
    ax.grid(True)

def save_technical_analysis_plot(df, output_path='/Users/benpfeffer/trading-icona-capital/trading-icona-capital/src/db/figures', version=0):
    """Create and save comprehensive technical analysis plot"""
    # Create figure with subplots in a 2-column layout
    fig = plt.figure(figsize=(20, 15))
    gs = fig.add_gridspec(5, 1, height_ratios=[2, 1, 1, 1, 1], 
                         hspace=0.3)
    
    # Create main chart subplots in single column
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1], sharex=ax1)
    ax3 = fig.add_subplot(gs[2], sharex=ax1)
    ax4 = fig.add_subplot(gs[3], sharex=ax1)
    ax5 = fig.add_subplot(gs[4], sharex=ax1)
    
    # Generate each subplot
    plot_price_and_keltner(ax1, df)
    plot_macd(ax2, df)
    plot_rsi(ax3, df)
    plot_moving_averages(ax4, df)
    plot_atr_variance(ax5, df)
    
    # Add title to the entire figure
    fig.suptitle('Technical Analysis Dashboard', fontsize=16, y=0.95)
    
    # Ensure output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Save plot with high resolution
    plt.savefig(f'{output_path}/{version}_technical_indicators.png', 
                bbox_inches='tight', dpi=300)
    
    plt.close()

# Generate and save the plot
save_technical_analysis_plot(df_, version=0)


In [14]:
"""
_Objective:
Determine whether the market (or individual stock) is in a trending or range-bound/high-volatility regime.

_Approach:
	•	Trending Regime:
	•	Lower ATR/variance combined with sustained directional movement (e.g., price above MA200, or a confirmed golden cross).
	•	Range-Bound/High-Volatility Regime:
	•	Higher ATR/variance values and oscillating price behavior.
"""





'\n_Objective:\nDetermine whether the market (or individual stock) is in a trending or range-bound/high-volatility regime.\n\n_Approach:\n\t•\tTrending Regime:\n\t•\tLower ATR/variance combined with sustained directional movement (e.g., price above MA200, or a confirmed golden cross).\n\t•\tRange-Bound/High-Volatility Regime:\n\t•\tHigher ATR/variance values and oscillating price behavior.\n'